### Import Library

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

### Load Dataset from API

In [2]:
baseurl = 'https://mrnrasyad.github.io/data/'
response = requests.get(baseurl)
response


<Response [200]>

In [3]:
#load dataset
retail_data1 = pd.read_json(baseurl + 'retail_data1.json',lines=True)
retail_data2 = pd.read_json(baseurl + 'retail_data2.json',lines=True)
retail_data3 = pd.read_json(baseurl + 'retail_data3.json',lines=True)
retail_data4 = pd.read_json(baseurl + 'retail_data4.json',lines=True)

### Concatenate multiple DataFrames, validate data types, checking duplicate and missing value

In [4]:
#Concat multiple dataframe menjadi 1 dataframe
retail_table = pd.concat([retail_data1, retail_data2, retail_data3, retail_data4])
print('\nJumlah baris:', retail_table.shape[0])
print('-- head')
display(retail_table.head())
print('-- tail')
display(retail_table.tail())
print('-- data type')
retail_table.info()


Jumlah baris: 50856
-- head


,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,total_price
0,1612885,01-01-19,16293,Malang,Jawa Timur,P1301,BRAND_F,6,747000,4482000
1,1612387,01-01-19,17228,Bogor,Jawa Barat,P2086,BRAND_L,4,590000,2360000
2,1612903,01-01-19,16775,Surakarta,Jawa Tengah,P1656,BRAND_G,3,1325000,3975000
3,1612963,01-01-19,0,unknown,unknown,P3127,BRAND_S,1,1045000,1045000
4,1612915,01-01-19,0,unknown,unknown,P1230,BRAND_E,1,-891000,891000


-- tail


,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,total_price
20178,1742263,31-12-19,0,unknown,unknown,P0471,BRAND_B,1,1164000,1164000
20179,1742266,31-12-19,14737,Yogyakarta,Yogyakarta,P0987,BRAND_D,12,590000,7080000
20180,1742923,31-12-19,14293,Bogor,Jawa Barat,P3487,BRAND_S,25,159000,3975000
20181,1742632,31-12-19,17250,Surabaya,Jawa Timur,P0036,BRAND_A,1,240000,240000
20182,1742629,31-12-19,14071,Bandung,Jawa Barat,P1108,BRAND_D,2,2900000,5800000


-- data type
<class 'pandas.core.frame.DataFrame'>
Index: 50856 entries, 0 to 20182
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_id     50856 non-null  object
 1   order_date   50856 non-null  object
 2   customer_id  50856 non-null  int64 
 3   city         50856 non-null  object
 4   province     50856 non-null  object
 5   product_id   50856 non-null  object
 6   brand        50856 non-null  object
 7   quantity     50856 non-null  int64 
 8   item_price   50856 non-null  int64 
 9   total_price  50856 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 4.3+ MB


In [5]:
#Cek duplikat
retail_table.duplicated().sum()

#Cek missing value
retail_table.isnull().sum()

,0
order_id,0
order_date,0
customer_id,0
city,0
province,0
product_id,0
brand,0
quantity,0
item_price,0
total_price,0


### Checking and handling negative `item_price` values

In [6]:
# Periksa nilai negatif pada item_price
negative_price_count = retail_table[retail_table['item_price'] < 0].shape[0]

if negative_price_count > 0:
    print(f"Found {negative_price_count} rows with negative 'item_price'. Removing these rows.")
    retail_table = retail_table[retail_table['item_price'] >= 0]
    print(f"Shape of retail_table after cleaning: {retail_table.shape}")
else:
    print("No negative 'item_price' values found.")

print(retail_table.head)

Found 2 rows with negative 'item_price'. Removing these rows.
Shape of retail_table after cleaning: (50854, 10)
<bound method NDFrame.head of       order_id order_date  customer_id        city     province product_id  \
0      1612885   01-01-19        16293      Malang   Jawa Timur      P1301   
1      1612387   01-01-19        17228       Bogor   Jawa Barat      P2086   
2      1612903   01-01-19        16775   Surakarta  Jawa Tengah      P1656   
3      1612963   01-01-19            0     unknown      unknown      P3127   
5      1612999   01-01-19            0     unknown      unknown      P0315   
...        ...        ...          ...         ...          ...        ...   
20178  1742263   31-12-19            0     unknown      unknown      P0471   
20179  1742266   31-12-19        14737  Yogyakarta   Yogyakarta      P0987   
20180  1742923   31-12-19        14293       Bogor   Jawa Barat      P3487   
20181  1742632   31-12-19        17250    Surabaya   Jawa Timur      P0036   


### Checking and handling undefined `order_id` values

In [7]:
# Cek apakah masih ada order_id yang bernilai undefined dan delete row tersebut
cek = retail_table.loc[retail_table['order_id'] == 'undefined']
print('\norder_id yang bernilai undefined:\n', cek)

# Jika ada maka buang baris tersebut
if cek.shape[0] != 0:
  retail_table = retail_table.loc[retail_table['order_id'] != 'undefined']


order_id yang bernilai undefined:
     order_id order_date  customer_id           city     province product_id  \
9  undefined   01-07-19        15150  Jakarta Pusat  DKI Jakarta      P3694   

     brand  quantity  item_price  total_price  
9  BRAND_T        12      159000      1908000  


### Changing `order_id` and `order_date` data type



In [8]:
# Transform order_id menjadi int64
retail_table['order_id'] = retail_table['order_id'].astype('int64')

# Transform order_date menjadi datetime Pandas
retail_table['order_date'] = pd.to_datetime(retail_table['order_date'])

# Cek dataframe info kembali untuk memastikan
print('\nInfo:')
print(retail_table.info())


Info:
<class 'pandas.core.frame.DataFrame'>
Index: 50853 entries, 0 to 20182
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     50853 non-null  int64         
 1   order_date   50853 non-null  datetime64[ns]
 2   customer_id  50853 non-null  int64         
 3   city         50853 non-null  object        
 4   province     50853 non-null  object        
 5   product_id   50853 non-null  object        
 6   brand        50853 non-null  object        
 7   quantity     50853 non-null  int64         
 8   item_price   50853 non-null  int64         
 9   total_price  50853 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 4.3+ MB
None


/tmp/ipython-input-1075695029.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  retail_table['order_date'] = pd.to_datetime(retail_table['order_date'])


### Handle `customer_id` with value `0` based on specific conditions

In [9]:
print(f"Shape of retail_table before conditional handling of customer_id = 0: {retail_table.shape}")

# Define the conditions for deletion
condition_to_delete = (
    (retail_table['customer_id'] == 0) &
    (retail_table['product_id'] == 'unknown') &
    (retail_table['brand'] == 'unknown')
)

# Count rows that meet the deletion conditions
conditional_rows_count = retail_table[condition_to_delete].shape[0]

if conditional_rows_count > 0:
    print(f"Found {conditional_rows_count} rows with customer_id = 0 AND product_id = 'unknown' AND brand = 'unknown'. Removing these rows.")
    # Remove rows that meet the conditions
    retail_table = retail_table.drop(retail_table[condition_to_delete].index)
    print(f"Shape of retail_table after removal: {retail_table.shape}")
else:
    print("No rows found matching the specified conditions.")

# Display head to confirm changes
display(retail_table.head())

Shape of retail_table before conditional handling of customer_id = 0: (50853, 10)
Found 54 rows with customer_id = 0 AND product_id = 'unknown' AND brand = 'unknown'. Removing these rows.
Shape of retail_table after removal: (50651, 10)


,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,total_price
0,1612885,2019-01-01,16293,Malang,Jawa Timur,P1301,BRAND_F,6,747000,4482000
1,1612387,2019-01-01,17228,Bogor,Jawa Barat,P2086,BRAND_L,4,590000,2360000
2,1612903,2019-01-01,16775,Surakarta,Jawa Tengah,P1656,BRAND_G,3,1325000,3975000
3,1612963,2019-01-01,0,unknown,unknown,P3127,BRAND_S,1,1045000,1045000
5,1612999,2019-01-01,0,unknown,unknown,P0315,BRAND_B,1,1188000,1188000


### Import and download cleaned **retail_table.csv** file

In [10]:
retail_table.to_csv('retail_table_cleaned.csv', index=False)

In [11]:
print('\nInfo:')
print(retail_table.info())


Info:
<class 'pandas.core.frame.DataFrame'>
Index: 50651 entries, 0 to 20182
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     50651 non-null  int64         
 1   order_date   50651 non-null  datetime64[ns]
 2   customer_id  50651 non-null  int64         
 3   city         50651 non-null  object        
 4   province     50651 non-null  object        
 5   product_id   50651 non-null  object        
 6   brand        50651 non-null  object        
 7   quantity     50651 non-null  int64         
 8   item_price   50651 non-null  int64         
 9   total_price  50651 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 4.3+ MB
None
